In [8]:
import os

import numpy as np
import skimage.io
import skimage.measure

from chromrings import data_path

In [6]:
images_path = os.path.join(
    data_path, '0_test_3D_Nada_24-11-2022', 'fed', 'Position_8', 'Images'
)

segm_filename = 'DCW153-fed -_s08_segm_fed.npz'
image_filename = 'DCW153-fed -_s08_w1SDC488.tif'

segm_filepath = os.path.join(images_path, segm_filename)
image_filepath = os.path.join(images_path, image_filename)

In [7]:
segm_data = np.load(segm_filepath)['arr_0']
img_data = skimage.io.imread(image_filepath)

In [17]:
%matplotlib qt

from IPython.core.debugger import set_trace
import matplotlib.pyplot as plt

rp = skimage.measure.regionprops(segm_data, intensity_image=img_data)
angle_range = (0, 360)
for obj in rp:
    zc, yc, xc = obj.weighted_centroid
    # get center z-slice
    x1, y1 = round(xc), round(yc)
    zmin, ymin, xmin, zmax, ymax, xmax = obj.bbox
    zc_local = zc-zmin
    z_local = round(zc_local)
    obj_z = obj.image[z_local]
    d, h, w = zmax-zmin, ymax-ymin, xmax-xmin  
    for angle in range(*angle_range):
        if angle == 0:
            half_w = int(w/2)+1
            rr, cc = [y1]*half_w, list(range(x1, x1+half_w))
            lab = segm_data[round(zc)].copy()
            lab[lab != obj.label] == 0
            plt.imshow(lab)
            plt.plot(cc, rr, 'r.')
            plt.show()
        elif angle == 90:
            half_h = int(h/2)+1
            rr, cc = list(range(y1, y1+half_h)), [x1]*half_h
        elif angle == 180:
            half_w = int(w/2)
            rr, cc = [y1]*(half_w+1), list(range(x1-half_w, x1+1))
        elif angle == 270:
            half_h = int(h/2)
            rr, cc = list(range(y1-half_h, y1+1)), [x1]*(half_h+1)

